In [2]:
#Handling Imports
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display

In [3]:
load_dotenv(override=True)

True

In [18]:
openai_api_key = os.getenv('OPENAI_API_KEY')
gemini_api_key = os.getenv('GOOGLE_API_KEY')
competitors = []
answers = []

In [19]:
request = "Please come up with a challenging, nuanced question that I can ask a number of LLMs to evaluate their intelligence. "
request += "Answer only with the question, no explanation."
messages = [{"role": "user", "content": request}]

In [20]:
openai = OpenAI()
response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages
)

question = response.choices[0].message.content


In [21]:
print(question)

How would you approach resolving a moral dilemma between individual rights and the greater good in a situation where both outcomes have significant but opposing impacts on a community?


In [22]:
messages = [
    {"role": "user", 
    "content": question}
]

In [24]:
model_name = "gpt-4o-mini"
response = openai.chat.completions.create(
    model=model_name,
    messages=messages
)

answer = response.choices[0].message.content

competitors.append(model_name)
answers.append(answer)


In [ ]:
display(Markdown(answer))

In [25]:
print(competitors)
print(answers)

['gpt-4o-mini']
['Resolving a moral dilemma between individual rights and the greater good is complex and requires careful consideration of several factors. Here are some steps to approach such a situation:\n\n1. **Define the Stakeholders**: Identify all individuals and groups impacted by the potential outcomes. Understanding their perspectives and interests is crucial.\n\n2. **Clarify the Rights Involved**: Examine the individual rights at stake. Consider why these rights are significant and what they entail. This step helps in recognizing the moral weight of protecting individual freedoms.\n\n3. **Assess the Greater Good**: Evaluate what "the greater good" means in this context. This can vary widely depending on the values and needs of the community. Determine who benefits and who might suffer, alongside the scope and duration of these impacts.\n\n4. **Gather Data and Perspectives**: Look for empirical evidence or case studies that relate to the dilemma. Understanding real-world cons

In [26]:
model_name = "gemini-2.0-flash"
gemini=OpenAI(api_key=gemini_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/")
gemini_response = gemini.chat.completions.create(
    model=model_name,
    messages=messages
)

gemini_answer = gemini_response.choices[0].message.content

competitors.append(model_name)
answers.append(gemini_answer)


In [28]:
print(competitors)
print(answers)
print(len(competitors))
print(len(answers))

['gpt-4o-mini', 'gemini-2.0-flash']
['Resolving a moral dilemma between individual rights and the greater good is complex and requires careful consideration of several factors. Here are some steps to approach such a situation:\n\n1. **Define the Stakeholders**: Identify all individuals and groups impacted by the potential outcomes. Understanding their perspectives and interests is crucial.\n\n2. **Clarify the Rights Involved**: Examine the individual rights at stake. Consider why these rights are significant and what they entail. This step helps in recognizing the moral weight of protecting individual freedoms.\n\n3. **Assess the Greater Good**: Evaluate what "the greater good" means in this context. This can vary widely depending on the values and needs of the community. Determine who benefits and who might suffer, alongside the scope and duration of these impacts.\n\n4. **Gather Data and Perspectives**: Look for empirical evidence or case studies that relate to the dilemma. Understan

In [29]:
together = ""
for index, answer in enumerate(answers):
    together += f"# Response from competitor {index+1}\n\n"
    together += answer + "\n\n"

In [ ]:
print(together)

In [31]:
judge = f"""You are judging a competition between {len(competitors)} competitors.
Each model has been given this question:

{question}

Your job is to evaluate each response for clarity and strength of argument, and rank them in order of best to worst.
Respond with JSON, and only JSON, with the following format:
{{"results": ["best competitor number", "second best competitor number", "third best competitor number", ...]}}

Here are the responses from each competitor:

{together}

Now respond with the JSON with the ranked order of the competitors, nothing else. Do not include markdown formatting or code blocks."""

In [40]:
model_name = "o3"
messages = [
    {"role": "user", 
    "content": judge}
]



In [41]:
response = openai.chat.completions.create(
    model=model_name,
    messages=messages
)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
result = response.choices[0].message.content

In [39]:
print(result)

{"results": ["2", "1"]}


In [43]:
results_dict = json.loads(result)
ranks = results_dict["results"]
for index, result in enumerate(ranks):
    competitor = competitors[int(result)-1]
    print(f"Rank {index+1}: {competitor}")

Rank 1: gemini-2.0-flash
Rank 2: gpt-4o-mini


In [46]:
# Now judging using gemini model
model_name = "gemini-2.0-flash"
gemini=OpenAI(api_key=gemini_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/")
gemini_response = gemini.chat.completions.create(
    model=model_name,
    messages=messages
)
result = gemini_response.choices[0].message.content

In [47]:
results_dict = json.loads(result)
ranks = results_dict["results"]
for index, result in enumerate(ranks):
    competitor = competitors[int(result)-1]
    print(f"Rank {index+1}: {competitor}")

Rank 1: gemini-2.0-flash
Rank 2: gpt-4o-mini
